In [4]:
# auto reload module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.feature_selection import SelectFromModel


from sklearn.model_selection import LeaveOneGroupOut
# LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score

from utils import load_dataset, flatten_data, select_emotions, train_test_split

In [18]:
data, labels, groups = load_dataset('all_features_4sec_epoch.npy')
data, labels, groups = flatten_data(data, labels, groups)
# data, labels, groups = select_emotions(data, labels, groups, 0, 2)
print(f"data shape: {data.shape}, labels shape: {labels.shape}, groups shape: {groups.shape}")

data shape: (10800, 2480), labels shape: (10800,), groups shape: (10800,)


In [19]:
X, y, groups = data, labels, groups

# Standardize the data
# Standardize the data
scaler = StandardScaler()

# Logistic Regression
log_reg = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Recursive feature elimination (RFE) using Logistic Regression
selector = SelectFromModel(log_reg, threshold='mean')

# Create a pipeline that first scales the data, then performs feature selection, and finally classifies with Logistic Regression
pipeline = Pipeline([
    ('scaler', scaler),
    ('log_reg', log_reg)
])

# Leave-One-Group-Out cross-validation
logo = LeaveOneGroupOut()

accuracies = []

for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Fit the pipeline
    pipeline.fit(X_train, y_train)

    # Predict on the test set
    y_pred = pipeline.predict(X_test)

    # Evaluate the performance
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:.4f}')
    accuracies.append(acc)

# Calculate the mean accuracy and AUC
mean_accuracy = np.mean(accuracies)

print(f'Mean Accuracy: {mean_accuracy:.4f}')
